In [1]:
import glob
import pandas as pd
df = pd.concat([pd.read_csv(f, encoding='utf-8', header=0) for f in glob.glob('Data/JC*.csv')],
    ignore_index=True, sort=False)

In [2]:
df.head(5)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,...,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,61.0,2015-09-21 14:53:16,2015-09-21 14:54:17,3185.0,City Hall,40.717733,-74.043845,3185.0,City Hall,40.717733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,290.0,2015-09-21 14:55:59,2015-09-21 15:00:50,3183.0,Exchange Place,40.716247,-74.033459,3187.0,Warren St,40.721124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,786.0,2015-09-21 14:56:14,2015-09-21 15:09:21,3183.0,Exchange Place,40.716247,-74.033459,3183.0,Exchange Place,40.716247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,477.0,2015-09-21 14:58:35,2015-09-21 15:06:32,3203.0,Hamilton Park,40.727596,-74.044247,3203.0,Hamilton Park,40.727596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,451.0,2015-09-21 14:59:55,2015-09-21 15:07:27,3203.0,Hamilton Park,40.727596,-74.044247,3186.0,Grove St PATH,40.719586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.count()

Trip Duration               339620
Start Time                  339620
Stop Time                   339620
Start Station ID            339620
Start Station Name          339620
Start Station Latitude      339620
Start Station Longitude     339620
End Station ID              339620
End Station Name            339620
End Station Latitude        339620
End Station Longitude       339620
Bike ID                     339620
User Type                   339123
Birth Year                  313288
Gender                      339620
tripduration               1040634
starttime                  1040634
stoptime                   1040634
start station id           1040634
start station name         1040634
start station latitude     1040634
start station longitude    1040634
end station id             1040634
end station name           1040634
end station latitude       1040634
end station longitude      1040634
bikeid                     1040634
usertype                   1040634
birth year          

In [4]:
df.columns

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender', 'tripduration', 'starttime', 'stoptime',
       'start station id', 'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [5]:
#Gotta combine rows from identical columns, however pandas requires this to be done from two dataframes, not one
#Therefore make two seperate dataframes, loc, then change column names back to match for append
df1_columns= ['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender']
df2_columns=['tripduration', 'starttime', 'stoptime',
       'start station id', 'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender']

df1 = df.loc[:,df1_columns]
df2 = df.loc[:,df2_columns]
df1 = df1.dropna(how="any")
df2 = df2.dropna(how="any")
df2.columns = df2.columns[:0].tolist() + df1_columns

In [6]:
useful_data = df1.append(df2,sort=False)

In [7]:
useful_data.count()

Trip Duration              1335515
Start Time                 1335515
Stop Time                  1335515
Start Station ID           1335515
Start Station Name         1335515
Start Station Latitude     1335515
Start Station Longitude    1335515
End Station ID             1335515
End Station Name           1335515
End Station Latitude       1335515
End Station Longitude      1335515
Bike ID                    1335515
User Type                  1335515
Birth Year                 1335515
Gender                     1335515
dtype: int64

In [8]:
#filter out "JSQ Don't Use", "WS Don't Use", etc. and malfunctioning coords
useful_data = useful_data.loc[(useful_data["End Station Latitude"] != 0.0) & (useful_data["End Station Name"] != "Hs Don't Use") & (useful_data["End Station Name"] != "Forrest Tech Shop (parts testing)") ,:]

In [9]:
useful_data.count()

Trip Duration              1334265
Start Time                 1334265
Stop Time                  1334265
Start Station ID           1334265
Start Station Name         1334265
Start Station Latitude     1334265
Start Station Longitude    1334265
End Station ID             1334265
End Station Name           1334265
End Station Latitude       1334265
End Station Longitude      1334265
Bike ID                    1334265
User Type                  1334265
Birth Year                 1334265
Gender                     1334265
dtype: int64

In [10]:
#Translate numerical gender code for visualization
gender_type ={0: 'Undeclared' , 1: 'Male' , 2: 'Female'} 
useful_data['Gender'] = useful_data['Gender'].map(gender_type)

In [11]:
# lat1=useful_data['Start Station Latitude']
# lon1=useful_data['Start Station Longitude']
# lat2=useful_data['End Station Latitude']
# lon2=useful_data['End Station Longitude']
import numpy as np
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

# dftest['dist']
useful_data['Displacement (km)'] = \
    haversine(useful_data['Start Station Latitude'], useful_data['Start Station Longitude'],
                 useful_data['End Station Latitude'], useful_data['End Station Longitude'])

In [12]:
useful_data['average velocity (km/h)']=useful_data['Displacement (km)']/(useful_data['Trip Duration']/3600)

In [119]:
#useful_data['average velocity (m/s)']=(useful_data['Displacement (km)']*1000)/useful_data['Trip Duration']

In [13]:
# tripdurations = useful_data.loc[(useful_data["Trip Duration"] > 60),:]
# tripdurations.sort_values(by='Trip Duration')
# this shows all trip durations are greater than a minute, therefore convert trip duration to minutes
useful_data['Trip Duration']=useful_data['Trip Duration']/60
useful_data = useful_data.rename(columns={"Trip Duration": "Trip Duration (min)"})

In [14]:
useful_data.to_csv("Data/citibike_complete.csv")

In [15]:
#Note there is one case of someone driving a car with a bike onboard
useful_data.sort_values(by='average velocity (km/h)', ascending=False)

,Trip Duration (min),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Displacement (km),average velocity (km/h)
1075644,4.516667,2019-05-27 18:37:03.6240,2019-05-27 18:41:35.4150,3186.0,Grove St PATH,40.719586,-74.043117,487.0,E 20 St & FDR Drive,40.733143,-73.975739,29300.0,Subscriber,1988.0,Male,5.874513,78.037817
1027169,2.166667,2019-04-16 04:12:24.9470,2019-04-16 04:14:35.7210,3206.0,Hilltop,40.731169,-74.057574,3209.0,Brunswick St,40.724176,-74.050656,29492.0,Subscriber,1987.0,Male,0.971745,26.909862
41033,1.533333,2015-11-30 22:53:28,2015-11-30 22:55:00,3195.0,Sip Ave,40.730743,-74.063784,3194.0,McGinley Square,40.725340,-74.067622,24674.0,Subscriber,1986.0,Male,0.682284,26.698057
21260,1.550000,2015-10-23 19:45:40,2015-10-23 19:47:13,3195.0,Sip Ave,40.730743,-74.063784,3194.0,McGinley Square,40.725340,-74.067622,24689.0,Subscriber,1986.0,Male,0.682284,26.410981
49156,1.566667,2015-12-19 00:44:47,2015-12-19 00:46:22,3195.0,Sip Ave,40.730743,-74.063784,3194.0,McGinley Square,40.725340,-74.067622,24484.0,Subscriber,1986.0,Male,0.682284,26.130013
493718,2.266667,2017-09-07 13:23:00,2017-09-07 13:25:17,3206.0,Hilltop,40.731169,-74.057574,3209.0,Brunswick St,40.724176,-74.050656,29569.0,Subscriber,1979.0,Male,0.971745,25.722662
38476,1.600000,2015-11-23 22:53:53,2015-11-23 22:55:29,3195.0,Sip Ave,40.730743,-74.063784,3194.0,McGinley Square,40.725340,-74.067622,24669.0,Subscriber,1986.0,Male,0.682284,25.585638
209740,3.333333,2016-01-17 11:50:50,2016-01-17 11:54:11,3220.0,5 Corners Library,40.734961,-74.059503,3209.0,Brunswick St,40.724176,-74.050656,24466.0,Subscriber,1981.0,Male,1.411998,25.415960
532929,2.300000,2017-10-10 23:02:42,2017-10-10 23:05:01,3206.0,Hilltop,40.731169,-74.057574,3209.0,Brunswick St,40.724176,-74.050656,31747.0,Subscriber,1981.0,Male,0.971745,25.349870
160719,6.233333,2016-08-23 08:50:01,2016-08-23 08:56:16,3206.0,Hilltop,40.731169,-74.057574,3183.0,Exchange Place,40.716247,-74.033459,24402.0,Subscriber,1981.0,Female,2.623515,25.253084


In [1]:
# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot

# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()